In [1]:
from functools import reduce
import pandas as pd

# Purpose
Dataverse ships with two metadata fields where depositors can enter the names of funders: In the Funding Information Agency field and in the Contributor field when they choose the Contributor Type "Funder".

Who uses these fields and why? We need to learn if the design of the fields are meeting some need that we weren't aware of and if we can re-design them to improve how funder metadata is used to describe who funds data and to search for data based on funders.

In this notebooks we'll be looking at the metadata of the latest versions of published datasets in most of the known Dataverse installations. Looking only at the metadata of the latest versions of each dataset might cause us to miss some things, such as under counting of the number of times people have used either or both fields to enter funder names. For example, the first version of a dataset might have both fields filled, but the latest version might have only one because a curator decided to use just one. By considering only the latest version of each dataset, we'll miss cases like this. But considering the metadata of only the latest published versions should help simplify this review.

# Prepare the data

In [2]:
# Import CSV file that lists PIDs and publication dates of all datasets, which Dataverse installation
# they're published in, which Dataverse collections they're publishing in, and the type of collection
datasetPIDsDF = (pd
    .read_csv(
        filepath_or_buffer='dataset_pids_from_most_known_dataverse_installations_2023.08.csv',
        usecols=lambda x: x not in ['dataset_pid', 'dataverse_name'],
        sep=',', na_filter=False)
    # Remove datasets whose metadata couldn't be saved and datasets from ODISSEI_Portal, since that installation only indexes metadata from datasets that are published in other repositories
    .query(
        '(dataverse_json_export_saved == True) and\
        (dataverse_installation_name != "ODISSEI_Portal")')
    .drop(columns=['dataverse_json_export_saved'])
    .reset_index(drop=True, inplace=False)
)

datasetPIDsDF.head()

,dataverse_installation_name,dataset_pid_url,dataverse_collection_alias,dataverse_collection_name,dataverse_collection_type
0,UNB_Libraries_Dataverse,https://doi.org/10.25545/NVN79Z,snb,Service New Brunswick,ORGANIZATIONS_INSTITUTIONS
1,UNB_Libraries_Dataverse,https://doi.org/10.25545/YB60PU,blightchemistry,Blight Chemistry Research Omniverse,RESEARCH_GROUP
2,UNB_Libraries_Dataverse,https://doi.org/10.25545/WHD3KC,blightchemistry,Blight Chemistry Research Omniverse,RESEARCH_GROUP
3,UNB_Libraries_Dataverse,https://doi.org/10.25545/6TZWCG,Ir-CationBinding,Elucidation of Charge Contributions in Iridium-Chelated Hydrogen-Bonding Systems,JOURNALS
4,UNB_Libraries_Dataverse,https://doi.org/10.25545/NYM13B,IFMLAB,Integrated Forest Management Lab,RESEARCH_GROUP


In [3]:
# Import funder metadata and remove the dataset_pid column
grantInformationMetadataDF = pd.read_csv(
    filepath_or_buffer='funding_information(citation)_2023.08.22-2023.08.28.csv',
    usecols=lambda x: x not in ['dataset_pid'],
    na_filter=False,
    sep=',')

print(f'Count of distinct dataset PIDs in grantInformationMetadataDF: {len(pd.unique(grantInformationMetadataDF["dataset_pid_url"]))}')

grantInformationMetadataDF.head()

Count of distinct dataset PIDs in grantInformationMetadataDF: 114280


,dataset_pid_url,dataset_publication_date,dataset_version_number,dataset_version_create_time,grantNumberAgency,grantNumberValue
0,https://doi.org/10.7910/DVN/QD0V0H,2021-09-20,1.0,2021-07-29T19:45:43Z,The Leon Levy Foundation,
1,https://doi.org/10.17026/dans-xhp-2h3w,2019-10-02,1.0,2022-01-31T03:58:32Z,NWO,380-60-007
2,https://doi.org/10.7910/DVN/FHUNGL,2021-10-19,1.0,2021-08-10T15:31:09Z,The Leon Levy Foundation,
3,https://hdl.handle.net/11272.1/AB2/ZZMOPP,2019-05-15,1.0,2020-08-21T23:23:32Z,Air Force Research Laboratory and Defense Advance Research Projects Agency,Air Force Research Laboratory and Defense Advance Research Projects Agency: FA8750-13-2-0045
4,https://doi.org/10.17026/dans-xxz-qvjx,2020-06-20,1.0,2022-01-29T06:23:21Z,NWO,380-60-007


In [4]:
# Import contributor metadata, remove the dataset_pid column and include only contributors who are the "Funder" contributor type
contributorMetadataDF = (pd
    .read_csv(
        filepath_or_buffer='contributor(citation)_2023.08.22-2023.08.28.csv',
        usecols=lambda x: x not in ['dataset_pid'],
        na_filter=False,
        sep=',')
    .query('(contributorType == "Funder")')
    .reset_index(drop=True, inplace=False))

print(f'Count of distinct dataset PIDs in contributorMetadataDF: {len(pd.unique(contributorMetadataDF["dataset_pid_url"]))}')

contributorMetadataDF.head()

Count of distinct dataset PIDs in contributorMetadataDF: 8655


,dataset_pid_url,dataset_publication_date,dataset_version_number,dataset_version_create_time,contributorType,contributorName
0,https://doi.org/10.5683/SP3/6FFGZN,2021-11-04,1.2,2021-12-09T18:34:39Z,Funder,Compute Ontario
1,https://doi.org/10.5683/SP3/XCS9W2,2021-11-04,1.2,2021-12-10T15:12:54Z,Funder,Compute Ontario
2,https://doi.org/10.5683/SP3/N4FNRC,2021-11-04,1.2,2021-12-09T14:49:06Z,Funder,Compute Ontario
3,https://doi.org/10.5683/SP3/DGXLXB,2021-11-04,1.2,2021-12-10T15:55:51Z,Funder,Compute Ontario
4,https://doi.org/10.5683/SP3/4WSVNS,2021-11-04,1.2,2021-12-08T21:45:32Z,Funder,Compute Ontario


In [5]:
# Join grantInformationMetadataDF and contributorMetadataDF
funderAndContributorFunderMetadataDF = (pd
    .merge(
        left=contributorMetadataDF,
        right=grantInformationMetadataDF,
        how='outer',
        on=['dataset_pid_url', 'dataset_publication_date', 'dataset_version_number', 'dataset_version_create_time'])
    .reset_index(drop=True, inplace=False))

funderAndContributorFunderMetadataDF.head()

,dataset_pid_url,dataset_publication_date,dataset_version_number,dataset_version_create_time,contributorType,contributorName,grantNumberAgency,grantNumberValue
0,https://doi.org/10.5683/SP3/6FFGZN,2021-11-04,1.2,2021-12-09T18:34:39Z,Funder,Compute Ontario,NaN,NaN
1,https://doi.org/10.5683/SP3/XCS9W2,2021-11-04,1.2,2021-12-10T15:12:54Z,Funder,Compute Ontario,NaN,NaN
2,https://doi.org/10.5683/SP3/N4FNRC,2021-11-04,1.2,2021-12-09T14:49:06Z,Funder,Compute Ontario,NaN,NaN
3,https://doi.org/10.5683/SP3/DGXLXB,2021-11-04,1.2,2021-12-10T15:55:51Z,Funder,Compute Ontario,NaN,NaN
4,https://doi.org/10.5683/SP3/4WSVNS,2021-11-04,1.2,2021-12-08T21:45:32Z,Funder,Compute Ontario,NaN,NaN


In [6]:
# Join datasetPIDsDF and funderAndContributorFunderMetadataDF
allFunderNameDataDF = (pd
    .merge(
        left=datasetPIDsDF,
        right=funderAndContributorFunderMetadataDF,
        how='inner',
        on=['dataset_pid_url'])
    .reset_index(drop=True, inplace=False))

allFunderNameDataDF.head()

,dataverse_installation_name,dataset_pid_url,dataverse_collection_alias,dataverse_collection_name,dataverse_collection_type,dataset_publication_date,dataset_version_number,dataset_version_create_time,contributorType,contributorName,grantNumberAgency,grantNumberValue
0,UNB_Libraries_Dataverse,https://doi.org/10.25545/ZXDJZ7,MarsLab,Mars Science Laboratory at UNB,LABORATORY,2020-09-12,1.0,2020-09-10T18:59:57Z,NaN,NaN,Canadian Space Agency,
1,Harvard_Dataverse,https://doi.org/10.7910/DVN/RXBNCZ,0000-0002-0068-6933,IgboSynCorp Dataverse,RESEARCH_GROUP,2022-07-08,1.0,2022-07-04T22:48:06Z,Funder,Lacuna Fund,"Meridian Institute, 105 Village Place, Dillion, Colorado 80435, United States of America",2020-02--3075551051
2,Harvard_Dataverse,https://doi.org/10.7910/DVN/YB9FWK,0000-0002-0068-6933,IgboSynCorp Dataverse,RESEARCH_GROUP,2022-07-07,1.0,2022-06-21T06:21:20Z,NaN,NaN,Lacuna Fund of the Meridian Institute,2020-02--3075551051
3,Harvard_Dataverse,https://doi.org/10.7910/DVN/EPP4MA,2001_US_race_crime_survey,2001 US Race & Crime Survey Dataverse,RESEARCH_PROJECTS,2015-06-18,1.0,2015-06-18T22:55:51Z,NaN,NaN,National Science Foundation,9906346
4,Harvard_Dataverse,https://doi.org/10.7910/DVN/97Q2B8,3EA-Lebanon,3EA Lebanon Impact Data,RESEARCH_PROJECTS,2022-01-31,1.1,2023-03-02T14:59:48Z,Funder,Dubai Cares,NaN,NaN


In [7]:
# Export dataframe to a CSV file
# allDataDF.to_csv('allFunderNameDataDF.csv', index=False)

## Exploring the data

Now that we've got the funding metadata of many datasets in known Dataverse repositories, as of late August 2023, let's start answering our questions. How many datasets in each installation have funder metadata in either or both of the fields?

In [8]:
# Let's save the queries we'll be using later to find datasets that have metadata in either or both fields
allRowsQuery = 'tuple()'
onlyFunderAgencyNameQuery = 'grantNumberAgency == grantNumberAgency and contributorName.isnull()'
onlyContributorQuery = '(contributorType == "Funder" and contributorName == contributorName) and\
    grantNumberAgency.isnull()'
bothFieldsQuery = '(grantNumberAgency == grantNumberAgency and grantNumberAgency != "") and\
        (contributorType == "Funder" and contributorName == contributorName)'

# Function for creating a dataframe with counts of dataset based on given query and groupings
def createCountsDF(dataframe, countsQuery, nameOfCountsColumn, groupBy, sortBy, ascendingSort):
    countsDataframe = (dataframe
        .query(countsQuery, engine='python')
        .drop(columns=dataframe.columns.difference(['dataset_pid_url', groupBy]))
        .drop_duplicates()
        .groupby([groupBy]).count()
        .rename(columns={'dataset_pid_url': nameOfCountsColumn})
        .sort_values(by=[sortBy], ascending=ascendingSort)
        .reset_index(drop=False, inplace=False))
    return countsDataframe

In [9]:
# Now let's use that function to create tables that show...

# Count of all datasets in each installation
countOfDatasetsPerInstallationDF = createCountsDF(
    dataframe=datasetPIDsDF,
    countsQuery=allRowsQuery,
    nameOfCountsColumn='count_of_datasets',
    groupBy='dataverse_installation_name',
    sortBy='dataverse_installation_name',
    ascendingSort=True)

# Count of datasets in each installations that have funder names in any of the two fields
countOfDatasetsWithAnyFunderMetadataPerInstallationDF = createCountsDF(
    dataframe=allFunderNameDataDF,
    countsQuery=allRowsQuery,
    nameOfCountsColumn='count_of_datasets_with_funder_names_either_field',
    groupBy='dataverse_installation_name',
    sortBy='dataverse_installation_name',
    ascendingSort=True)

# Count of datasets in each installations that have funder names only in the Contributor field,
countOfFunderNameinOnlyContributorFieldsDF = createCountsDF(
    dataframe=allFunderNameDataDF,
    countsQuery=onlyContributorQuery,
    nameOfCountsColumn='count_of_datasets_with_funders_names_only_contributor_field',
    groupBy='dataverse_installation_name',
    sortBy='dataverse_installation_name',
    ascendingSort=True)

# Count of datasets in each installations that have funder names only in the Funder Information field
countOfFunderNameinOnlyFunderAgencyNameFieldsDF = createCountsDF(
    dataframe=allFunderNameDataDF,
    countsQuery=onlyFunderAgencyNameQuery,
    nameOfCountsColumn='count_of_datasets_with_funder_names_only_funder_agency_name_field',
    groupBy='dataverse_installation_name',
    sortBy='dataverse_installation_name',
    ascendingSort=True)

# Count of datasets in each installations that have funder names in both fields
countOfFunderNameinBothFieldsDF = createCountsDF(
    dataframe=allFunderNameDataDF,
    countsQuery=bothFieldsQuery,
    nameOfCountsColumn='count_of_datasets_with_funder_names_both_fields',
    groupBy='dataverse_installation_name',
    sortBy='dataverse_installation_name',
    ascendingSort=True)

In [10]:
# Let's merge these five dataframes
dataframes = [
    countOfDatasetsPerInstallationDF,
    countOfDatasetsWithAnyFunderMetadataPerInstallationDF,
    countOfFunderNameinOnlyContributorFieldsDF,
    countOfFunderNameinOnlyFunderAgencyNameFieldsDF,
    countOfFunderNameinBothFieldsDF
]
indexList = ['dataverse_installation_name']
for df in dataframes:
    df.set_index(indexList, inplace=True)
allCountsDF = reduce(lambda left, right: left.join(right, how='outer'), dataframes)

# Format the data so that...
allCountsDF = (allCountsDF
    # NaN values are changed to 0
    .fillna(0)
    # Values in new count columns are integers instead of floats
    .astype({
        'count_of_datasets_with_funder_names_either_field': 'int',
        'count_of_datasets_with_funders_names_only_contributor_field': 'int',
        'count_of_datasets_with_funder_names_only_funder_agency_name_field': 'int',
        'count_of_datasets_with_funder_names_both_fields': 'int'})
    # Reset index
    .reset_index(drop=False, inplace=False))

# # Let's add a column that shows the percentage of all datasets that have funder metadata either field
# countOfDatasetsTotalAndWithFunderMetadataDF['percentage_with_funder_metadata'] = (
#     (
#         countOfDatasetsTotalAndWithFunderMetadataDF['count_of_datasets_with_funders'] /
#         countOfDatasetsTotalAndWithFunderMetadataDF['count_of_all_datasets'])*100).round(1)

allCountsDF.head()

# allCountsDF.to_csv('allCountsDF.csv', index=False)

,dataverse_installation_name,count_of_datasets,count_of_datasets_with_funder_names_either_field,count_of_datasets_with_funders_names_only_contributor_field,count_of_datasets_with_funder_names_only_funder_agency_name_field,count_of_datasets_with_funder_names_both_fields
0,ACSS_Dataverse,137,17,0,17,0
1,ADA_Dataverse,1648,753,512,95,90
2,ASU_Library_Research_Data_Repository,61,14,0,14,0
3,AUSSDA_Dataverse,1517,1497,0,1497,0
4,Abacus,2417,84,0,84,0


## Exploring installation's metadata
There are a few installations with datasets whose depositors have used the Contributor Name field more often than the Funder Information field and a few with datasets whose depositors have used both fields:

In [11]:
installationsWithMoreFundersInContributorName = (allCountsDF
    .query(
        'count_of_datasets_with_funders_names_only_contributor_field >\
        count_of_datasets_with_funder_names_only_funder_agency_name_field',
        engine='python')
    .reset_index(drop=True, inplace=False))

installationsWithMoreFundersInContributorName = installationsWithMoreFundersInContributorName[[
    'dataverse_installation_name',
    'count_of_datasets_with_funders_names_only_contributor_field',
    'count_of_datasets_with_funder_names_only_funder_agency_name_field']]

installationsWithMoreFundersInContributorName.head(10)

,dataverse_installation_name,count_of_datasets_with_funders_names_only_contributor_field,count_of_datasets_with_funder_names_only_funder_agency_name_field
0,ADA_Dataverse,512,95
1,Borealis,5913,911
2,CIMMYT_Research_Data,344,27
3,International_Potato_Center,587,1
4,MELDATA,339,0
5,PAPYRUS,12,5


In [12]:
installationsWithFundersInBothFields = (allCountsDF
    .query(
        'count_of_datasets_with_funder_names_both_fields > 0',
        engine='python')
    .sort_values(by=['count_of_datasets_with_funder_names_both_fields'], ascending=False)
    .reset_index(drop=True, inplace=False))

installationsWithFundersInBothFields = installationsWithFundersInBothFields[[
    'dataverse_installation_name',
    'count_of_datasets_with_funder_names_both_fields']]

installationsWithFundersInBothFields.head(5)

,dataverse_installation_name,count_of_datasets_with_funder_names_both_fields
0,Harvard_Dataverse,263
1,ADA_Dataverse,90
2,MELDATA,81
3,CIMMYT_Research_Data,58
4,Recherche_Data_Gouv,15


I've spoken with or started speaking with the managers of most of these installations to learn why the Contributor Name field has been used more than or instead of the Funding Information Name field or why both fields have been used. Let's get some more information about the funder name metadata in these installations.


### ADA Dataverse
In a GitHub issue comment at https://github.com/IQSS/dataverse/issues/4859#issuecomment-1573684412, a manager of ADA Dataverse, Steve McEachern, shared his reasoning for using the Contributor Name field more often. I've emailed Steve to ask follow up questions. Here's a table with the funder metadata from ADA Dataverse.

In [13]:
# Create dataframe for just metadata of datasets in ADA
datasetInADADF = (allFunderNameDataDF
    .query('dataverse_installation_name == "ADA_Dataverse"')
    .drop(columns=['dataverse_installation_name'])
    .reset_index(drop=True, inplace=False))

print(f'Count of datasets in ADA with funder metadata: {len(pd.unique(datasetInADADF["dataset_pid_url"]))}')
datasetInADADF.head()

Count of datasets in ADA with funder metadata: 753


,dataset_pid_url,dataverse_collection_alias,dataverse_collection_name,dataverse_collection_type,dataset_publication_date,dataset_version_number,dataset_version_create_time,contributorType,contributorName,grantNumberAgency,grantNumberValue
0,http://dx.doi.org/10.26193/00HBWG,ada-general-collection,ADA General Collection Dataverse,UNCATEGORIZED,2018-12-17,2.0,2019-05-13T08:43:01Z,Funder,"General Practice Evaluation Program, Department of Health, Housing and Community Services",NaN,NaN
1,http://dx.doi.org/10.26193/04F7C1,australia-at-work,Australia at Work Dataverse,UNCATEGORIZED,2019-01-25,2.0,2019-05-13T10:02:15Z,Funder,Australian Research Council,NaN,NaN
2,http://dx.doi.org/10.26193/0AF6TZ,bnla,Building a New Life in Australia Dataverse,RESEARCH_PROJECTS,2019-09-18,5.0,2022-02-02T23:01:26Z,Funder,Department of Social Services,NaN,NaN
3,http://dx.doi.org/10.26193/0AF6TZ,bnla,Building a New Life in Australia Dataverse,RESEARCH_PROJECTS,2019-09-18,5.0,2022-02-02T23:01:26Z,Funder,Department of Immigration and Border Protection,NaN,NaN
4,http://dx.doi.org/10.26193/0AGXC9,australian_historical_criminal_justice_data,Australian Historical Criminal Justice Data Dataverse,RESEARCH_PROJECTS,2020-05-12,1.0,2020-05-12T00:14:01Z,NaN,NaN,Australian Research Council,FL130100050


### Borealis

I've started talking with managers of Borealis to learn why depositors and curators there use the Contributor Name field more often. I think curation and publishing in Borealis is distributed, so it might be helpful to see which collections in Borealis have datasets that funder names in either of these fields and and which fields they're using most often. Then we'll know who to reach out to if we need to learn why particular collections have used the metadata fields in particular ways.

In [14]:
# Create dataframe for just metadata of datasets in ADA
datasetsInBorealisDF = (allFunderNameDataDF
                  .query('dataverse_installation_name == "Borealis"')
                  .drop(columns=['dataverse_installation_name'])
                  .reset_index(drop=True, inplace=False))

print(f'Count of datasets in Borealis with funder metadata: {len(pd.unique(datasetsInBorealisDF["dataset_pid_url"]))}')
datasetsInBorealisDF.head()

datasetsInBorealisDF.to_csv('datasetsInBorealisDF.csv', index=False)

Count of datasets in Borealis with funder metadata: 6828


In [15]:
# Now let's get counts of these datasets by collection
countOfFunderNamesinContributorNameFieldInBorealisDF = createCountsDF(
    dataframe=datasetsInBorealisDF,
    countsQuery=onlyContributorQuery,
    nameOfCountsColumn='count_of_datasets_with_funders_names_only_contributor_field',
    groupBy='dataverse_collection_alias',
    sortBy='count_of_datasets_with_funders_names_only_contributor_field',
    ascendingSort=False)

countOfFunderNamesinContributorNameFieldInBorealisDF.head()

,dataverse_collection_alias,count_of_datasets_with_funders_names_only_contributor_field
0,topomaps,5902
1,MDL,7
2,g360-projects,1
3,g360guelph,1
4,toronto,1


Almost all of the funder metadata in Borealis is coming from datasets in one collection. I've let the managers of Borealis know this in case it's helpful for figuring out why the dataset depositors used the Contributor Name field.

### Harvard Dataverse
- In the Harvard Dataverse Repository, which collections have datasets with the most funding metadata? This will help us figure out who to learn from when we make changes to how funding metadata is entered.
- Which funder agency names are entered most often? Knowing that might help us figure out how effective our efforts to standardize funder agency name metadata could be? For example, do the changes to the metadata fields (the "CV javascript") make it easier for depositors to enter the most popular funder agency names? How much easier?

In [16]:
# Lets start by creating a dataframe containing only metadata of datasets published in the Harvard Dataverse Repository
datasetsInHDVDF = (allFunderNameDataDF
    .query('(installation == "Harvard Dataverse")')
    .drop(columns=['installation'])
    .reset_index(drop=True, inplace=False)
    )

countOfDatasetsInHDVDF = len(pd.unique(datasetsInHDVDF['dataset_pid_url']))
print(f'Number of datasets in datasetInHDVDF: {countOfDatasetsInHDVDF}')

datasetsInHDVDF.head()

UndefinedVariableError: name 'installation' is not defined

So of the 80,278 datasets in the Harvard Dataverse Repository as of early October 2022, 30,222 had metadata about funding in one of the three fields where we expect it.

What's entered most often in the grantNumberAgency (Funding Information Name) field?

In [ ]:
# Now let's get counts of these datasets by collection
countOfFunderNamesinContributorNameFieldInHDVDF = createCountsDF(
    dataframe=datasetsInHDVDF,
    countsQuery=onlyContributorQuery,
    nameOfCountsColumn='count_of_datasets_with_funders_names_only_contributor_field',
    groupBy='dataverse_collection_alias',
    sortBy='count_of_datasets_with_funders_names_only_contributor_field',
    ascendingSort=False)

countOfFunderNamesinContributorNameFieldInHDVDF.head()

In [ ]:


# Create a new dataframe that lists each funder name entered in grantNumberAgency and the number of datasets with that funder name
datasetCountByFundingAgencyNamesInHDV = (
    datasetsInHDVDF[['dataset_pid_url', 'grantNumberAgency']]
        .query('grantNumberAgency == grantNumberAgency')
        .drop_duplicates()
        .groupby(['grantNumberAgency']).count()
        .rename(columns={'dataset_pid_url': 'count_of_datasets'})
        .sort_values(by=['count_of_datasets'], ascending=False)
        .reset_index(drop=False, inplace=False)
)

datasetCountByFundingAgencyNamesInHDV.head(10)

It's most likely that the Leon Levy collection has the most datasets with a funder name in their grantNumberAgency (Funding Information Name) field.

It also looks like, for datasets with other funder name metadata, we might have to deal with the variations in spellings, maybe using a deduplication or fuzzy matching library, in order to see which funders are added to the funding metadata most often. For example, this shows only 57 datasets with the value "NIH" in the Funding Information Agency field, but I've found more than 57 datasets created from NIH-funded research, with other values in the field, such as "National Institute of Health".

For now, let's see which collections have the most datasets with funding metadata, excluding the Leon Levy collections.

In [ ]:
datasetsInHDVByCollection_NoLevy = (
    datasetsInHDVDF[['dataset_pid_url', 'dataverse_alias']]
    .drop_duplicates()
    .groupby(['dataverse_alias']).count()
    .query('~dataverse_alias.str.contains("levy_photos").values')
    .rename(columns={'dataset_pid_url': 'count_of_datasets'})
    .sort_values(by=['count_of_datasets'], ascending=False)
    .reset_index(drop=False, inplace=False)
)

datasetsInHDVByCollection_NoLevy.head(10)

The IFPRI collections (IFPRI and AfricaRISING), worldfish collection and CIAT collection have the most datasets with funding metadata. ("harvard" is the alias of repository's main collection, where anyone can add data.)

What have the depositors of those datasets entered in the metadata?


In [ ]:
# fundingDatasetMetadataInHDVDF.to_csv('fundingDatasetMetadataInHDVDF.csv', index=False)

In [ ]:
def funder_names_in_collections(dataframe, collectionAliasesList):
    funderNamesInCollectionDF = (
        # Drop all but the needed columns
        dataframe[[
            'dataset_pid_url', 'grantNumberAgency', 'dataverse_alias']]
        # Remove all datasets except those in given list of collection aliases
        .query(
            'dataverse_alias in @collectionAliasesList')
        # Remove the 'dataverse_alias' column
        .drop(columns=['dataverse_alias'])
        # Keep only datasets that have funding agency name metadata
        .query('grantNumberAgency == grantNumberAgency')
        # Drop duplicate rows
        .drop_duplicates()
        # Group by funding agency name
        .groupby(['grantNumberAgency']).count()
        .rename(columns={'dataset_pid_url': 'count_of_datasets'})
        # Sort by count of occurrences of each funder name
        .sort_values(by=['count_of_datasets'], ascending=False)
        .reset_index(drop=False, inplace=False))
    return funderNamesInCollectionDF


In [ ]:
datasetCountByFundingAgencyNamesIFPRI = funder_names_in_collections(
    dataframe=datasetsInHDVDF,
    collectionAliasesList=['IFPRI', 'AfricaRISING'])

datasetCountByFundingAgencyNamesIFPRI.head()

In [ ]:
datasetCountByFundingAgencyNamesWorldfish = funder_names_in_collections(
    dataframe=datasetsInHDVDF,
    collectionAliasesList=['worldfish'])

datasetCountByFundingAgencyNamesWorldfish.head()

# funderNamesList_Worldfish = list(set(datasetCountByFundingAgencyNamesWorldfish["grantNumberAgency"].values.tolist()))

In [ ]:
datasetCountByFundingAgencyNamesCIAT = funder_names_in_collections(
    dataframe=datasetsInHDVDF,
    collectionAliasesList=[ # Aliases of collections I think are associated with CIAT
        'CIAT',
        'AgBio',
        'AICCRA',
        'CIFOR',
        'gender',
        'crp6',
        'dapa',
        'AllianceBioversityCIATFoodConsumer',
        'AllianceBioversityCIATLandscapes',
        'AllianceBioversityCIATClimate',
        'AllianceBioversityCIATBiodiversity',
        'AllianceBioversityCIATDigital',
        'AllianceBioversityCIATCrops4NH',
        'soils',
        'AllianceBioversityCIAT'
    ])

datasetCountByFundingAgencyNamesCIAT.head(20)

In [ ]:
datasetCountByFundingAgencyNamesDFEEP = funder_names_in_collections(
    dataframe=datasetsInHDVDF,
    collectionAliasesList=['DFEEP', 'ipa', 'jpal'])

datasetCountByFundingAgencyNamesDFEEP.head()

### Duplicate funding metadata in Harvard Dataverse

There are two fields where depositors can enter the names of funders: In the Funding Information Agency field and in the Contributor field when they choose the Contributor Type "Funder". How often has this happened and who's done it? By learning these things, we can see how big the problem is and who's used both fields? And later we can ask those people why. We need to learn if the design of the fields are meeting some need that we weren't aware of.

For now, let's continue looking only at the latest version of each dataset. This might cause a further under count of the number of times this issue has actually occurred. For example, first version of a dataset might have both fields filled but the latest might have only one. By considering only the latest version of each dataset, we'll miss cases like this. Eventually we'll have to think about what to do about the metadata of previous dataset versions.

Questions
- In the Harvard Dataverse Repository, how many datasets have values in the Funder Information fields and in the Contributor field when Contributor Type is "Funder"?
- How many datasets have funding metadata in their Contributor field and not in their Funder Information fields?
- How often are the same values in both fields? For example, one dataset might have "NIH" in the Funder Information Agency field and in the Contributor Name field when the Contributor Type is "Funder". How often do things like this happen?
- How often are different values in both fields?

In [ ]:
# Now let's query it to get only datasets that have values in both metadata fields
# Let's see which collections have most of these datasets


In [ ]:
datasetCountDuplicateFundingFields = len(pd.unique(duplicateFundingFieldsInHDV['dataset_pid_url']))
print(f'Number of datasets with metadata in both funding metadata fields: {datasetCountDuplicateFundingFields}')

In [ ]:
# duplicateFundingFieldsInHDV.to_csv('duplicateFundingFieldsInHDV.csv', index=False)

How many datasets have funding metadata in their Contributor field and not in their Grant Information fields?